In [1]:
# train_ds = data['train'].map(lambda x,y: (resize(x),y)).shuffle(1024).cache().batch(config.batch_size).prefetch(-1)
def get_hardest_k_examples(test_dataset, model, k=32):
    class_probs = model.predict(test_dataset)
    predictions = np.argmax(class_probs, axis=1)
    losses = tf.keras.losses.categorical_crossentropy(test_dataset.y, class_probs)
    argsort_loss =  np.argsort(losses)

    highest_k_losses = np.array(losses)[argsort_loss[-k:]]
    hardest_k_examples = test_dataset.x[argsort_loss[-k:]]
    true_labels = np.argmax(test_dataset.y[argsort_loss[-k:]], axis=1)

    return highest_k_losses, hardest_k_examples, true_labels, predictions
        
def log_high_loss_examples(test_dataset, model, k=32):
    print(f'logging k={k} hardest examples')
    losses, hardest_k_examples, true_labels, predictions = get_hardest_k_examples(test_dataset, model, k=k)
    wandb.log(
        {"high-loss-examples":
                            [wandb.Image(hard_example, caption = f'true:{label},\npred:{pred}\nloss={loss}')
                             for hard_example, label, pred, loss in zip(hardest_k_examples, true_labels, predictions, losses)]
        })

from IPython.display import display
import warnings
warnings.filterwarnings('ignore')
from pyleaves.utils import set_tf_config
set_tf_config(num_gpus=1)

import wandb
from wandb.keras import WandbCallback
# wandb.login()

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Conv2D, BatchNormalization, MaxPool2D, ReLU, ELU, LeakyReLU, Flatten, Dense, Add, AveragePooling2D, GlobalAveragePooling2D
from tensorflow.keras.layers.experimental.preprocessing import StringLookup, CategoryEncoding
import pprint
pp = pprint.PrettyPrinter(indent=4)

import numpy as np
np.random.seed(666)
tf.random.set_seed(666)

from typing import List, Tuple, Union
import tensorflow_datasets as tfds
from omegaconf import OmegaConf

# Which GPU is being used?
!nvidia-smi


setGPU: Setting GPU to: [0]
Initial visible GPUs: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
visible GPUs: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Successfully set memory_growth=True and limited GPUs visible to tensorflow.

Now using GPU(s):
['/physical_device:GPU:0']


/media/data/conda/jacob/envs/pyleaves2.3/lib/python3.7/site-packages/wandb/vendor/graphql-core-1.1/graphql/type/directives.py:55: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  assert isinstance(locations, collections.Iterable), 'Must provide locations for directive.'


Wed Nov 25 21:22:58 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  TITAN X (Pascal)    Off  | 00000000:04:00.0 Off |                  N/A |
| 23%   21C    P8     7W / 250W |     10MiB / 12196MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  TITAN X (Pascal)    Off  | 00000000:05:00.0 Off |                  N/A |
| 23%   23C    P8     8W / 250W |     10MiB / 12196MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   2  T

In [2]:
TFDS_DATASETS = ['plant_village']

class ClassLabelEncoder:
    def __init__(self, ds_info: tfds.core.dataset_info.DatasetInfo):
        self.info = ds_info
        self.dataset_name = ds_info.full_name
        self.num_samples = ds_info.splits['train'].num_examples
        self.num_classes = ds_info.features['label'].num_classes
        self.class_list = ds_info.features['label'].names
        self._str2int = ds_info.features['label'].str2int
        self._int2str = ds_info.features['label'].int2str
        
    def str2int(self, labels: Union[List[str],Tuple[str]]):
        labels = _valid_eager_tensor(labels)
        if not isinstance(labels, [list, tuple]):
            assert isinstance(labels, str)
            labels = [labels]
        return [self._str2int(l) for l in labels]
    
    def int2str(self, labels: Union[List[int],Tuple[int]]):
        labels = _valid_eager_tensor(labels)
        if not isinstance(labels, [list, tuple]):
            assert isinstance(labels, (int, np.int64))
            labels = [labels]
        return [self._int2str(l) for l in labels]
    
    def one_hot(self, label: tf.int64):
        '''
        One-Hot encode integer labels
        Use tf.data.Dataset.map(lambda x,y: (x, encoder.one_hot(y))) and pass in individual labels already encoded in int64 format.
        '''
        return tf.one_hot(label, depth=self.num_classes)
    
    def __repr__(self):
        return f'''Dataset Name: {self.dataset_name}
        Num_samples: {self.num_samples}
        Num_classes: {self.num_classes}'''
    
    def _valid_eager_tensor(self, tensor, strict=False):
        '''
        If tensor IS an EagerTensor, return tensor.numpy(). 
        if strict==True, and tensor IS NOT an EagerTensor, then raise AssertionError.
        if strict==False, and tensor IS NOT an EagerTensor, then return tensor without modification 
        '''
        try:
            assert isinstance(labels, tf.python.framework.ops.EagerTensor)
        except AssertionError:
            if strict:
                raise AssertionError(f'Strict EagerTensor requirement failed assertion test in ClassLabelEncoder._valid_eager_tensor method')
        labels = labels.numpy()
        return labels

def load_plant_village_dataset(split={'train':'train'},
                               data_dir=None,
                               batch_size=None):
    
    builder = tfds.builder(dataset_name, data_dir=data_dir)
    ds_info = builder.info
    builder.download_and_prepare()

    print(f'splits: {split}')
    data = builder.as_dataset(split=split,
                              shuffle_files=True,
                              batch_size=batch_size,
                              as_supervised=True
                              )
    
    return data, builder

def load_tfds_dataset(dataset_name='plant_village', 
                      split={'train':'train'},
                      data_dir=None,
                      batch_size=None):
    '''
    General interface function to properly route users to the correct function for loading their queried dataset from Tensorflow Datasets (TFDS) public data.
    '''
    assert dataset_name in TFDS_DATASETS
    
    print(f'Getting the TFDS dataset: {dataset_name}')
    if dataset_name == 'plant_village':
        return load_plant_village_dataset(split      =split,
                                          data_dir   =data_dir,
                                          batch_size =batch_size)
    else:
        raise Exception('Attempted to load dataset from TFDS that we have yet to build an adapter for. Consider building a minimal working prototype by using alternative datasets as a template.')
    

/media/data/conda/jacob/envs/pyleaves2.3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [3]:
from typing import Dict, Tuple, List
from tfrecord_utils.img_utils import resize_repeat
from boltons.funcutils import partial
# resize = resize_repeat(target_size=config.target_size, training=False)
def get_parse_example_func(target_size, class_encoder):
    resize = resize_repeat(target_size=target_size, training=False)
    def _parse_example(x, y):
        x = tf.image.convert_image_dtype(x, tf.float32)
        x = resize(x)
        y = class_encoder.one_hot(y)
        return x,y
#         y = tf.one_hot(y, depth=num_classes)
    return _parse_example

def preprocess_data(data: tf.data.Dataset, batch_size=1, target_size=None, class_encoder=None):
    parse_example = get_parse_example_func(target_size=target_size, class_encoder=class_encoder)
    return data.map(lambda x,y: parse_example(x, y)) \
                .shuffle(1024) \
                .batch(batch_size) \
                .prefetch(-1)

In [4]:
# def load_and_log(dataset_name='plant_village', 
#                  data_dir    = '/media/data/jacob/tensorflow_datasets'):
# dataset_name='plant_village'
# data_dir = '/media/data/jacob/tensorflow_datasets'
# if True:
#     split = {
#              'train': 'train[0%:60%]',
#              'val': 'train[60%:70%]',
#              'test': 'train[70%:100%]'
#             }
#     with wandb.init(project="artifacts-example", job_type="load-data") as run:
#     run   = wandb.init(project="artifacts-example", job_type="load-data")        
#     data, builder = load_tfds_dataset(dataset_name=dataset_name,
#                                       split=split,
#                                       data_dir=data_dir,
#                                       batch_size=None)
#     data_info     = builder.info
#     raw_data      = wandb.Artifact(
#                                     f"{dataset_name}-raw", type="dataset",
                                    
#                                     description="Raw {plant_village} dataset, split into train/val/test",
#                                     metadata={"source": "keras.datasets.mnist",
#                                               "sizes": [len(dataset.x) for dataset in datasets]})
# split = {
#          'train': 'train[0%:60%]',
#          'val': 'train[60%:70%]',
#          'test': 'train[70%:100%]'
#         }    
# data_dir = '/media/data/jacob/tensorflow_datasets'
# data, ds_info = load_plant_village_dataset(split=split,
#                                          data_dir=data_dir)

## Creating and tracking label encoders

In [4]:
dataset_name='plant_village'
data_dir = '/media/data/jacob/tensorflow_datasets'

split = {
         'train': 'train[0%:60%]',
         'val': 'train[60%:70%]',
         'test': 'train[70%:100%]'
        }

data, builder = load_tfds_dataset(dataset_name=dataset_name,
                                  split=split,
                                  data_dir=data_dir,
                                  batch_size=None)
data_info     = builder.info
class_encoder = ClassLabelEncoder(data_info)
print(class_encoder)
vocab = class_encoder.class_list

config = OmegaConf.create({'seed':237,
                           'batch_size':16,
                           'input_shape':(224,224,3),
                           'output_size':38,
                           'epochs_per_organism':3
                          })

preprocess = partial(preprocess_data,
                     batch_size=config.batch_size,
                     target_size=config.input_shape[:2],
                     class_encoder=class_encoder)

data['train'] = preprocess(data=data['train']) #, batch_size=config.batch_size)
data['val'] = preprocess(data=data['val']) #, batch_size=config.batch_size)
data['test'] = preprocess(data=data['test']) #, batch_size=config.batch_size)

Getting the TFDS dataset: plant_village
splits: {'train': 'train[0%:60%]', 'val': 'train[60%:70%]', 'test': 'train[70%:100%]'}
Dataset Name: plant_village/1.0.2
        Num_samples: 54303
        Num_classes: 38


In [6]:
# x, y = next(iter(train_ds))

import matplotlib.pyplot as plt
# plt.imshow(x[0,...])
# x.shape

# # Load the training and testing set of CIFAR10
# (X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()

# X_train = X_train.astype('float32')
# X_train = X_train/255.

# X_test = X_test.astype('float32')
# X_test = X_test/255.

# y_train = tf.reshape(tf.one_hot(y_train, 10), shape=(-1, 10))
# y_test = tf.reshape(tf.one_hot(y_test, 10), shape=(-1, 10))

# # Create TensorFlow dataset
# BATCH_SIZE = 256
# AUTOTUNE = tf.data.experimental.AUTOTUNE

# train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train))
# train_ds = train_ds.shuffle(1024).cache().batch(BATCH_SIZE).prefetch(AUTOTUNE)

# test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test))
# test_ds = test_ds.cache().batch(BATCH_SIZE).prefetch(AUTOTUNE)



# Organism
An organism contains the following:

1. phase - This denotes which phase does the organism belong to
2. chromosome - A dictionary of genes (hyperparameters)
3. model - The `tf.keras` model corresponding to the chromosome
4. prevBestOrganism - The best organism in the previous **phase**

In [5]:
options_phase0 = {
    'a_filter_size': [(1,1), (3,3), (5,5), (7,7), (9,9)],
    'a_include_BN': [True, False],
    'a_output_channels': [8, 16, 32, 64, 128, 256, 512],
    'activation_type': [ReLU, ELU, LeakyReLU],
    'b_filter_size': [(1,1), (3,3), (5,5), (7,7), (9,9)],
    'b_include_BN': [True, False],
    'b_output_channels': [8, 16, 32, 64, 128, 256, 512],
    'include_pool': [True, False],
    'pool_type': [MaxPool2D, AveragePooling2D],
    'include_skip': [True, False]
}

options = {
    'include_layer': [True, False],
    'a_filter_size': [(1,1), (3,3), (5,5), (7,7), (9,9)],
    'a_include_BN': [True, False],
    'a_output_channels': [8, 16, 32, 64, 128, 256, 512],
    'b_filter_size': [(1,1), (3,3), (5,5), (7,7), (9,9)],
    'b_include_BN': [True, False],
    'b_output_channels': [8, 16, 32, 64, 128, 256, 512],
    'include_pool': [True, False],
    'pool_type': [MaxPool2D, AveragePooling2D],
    'include_skip': [True, False]
}

In [6]:
class Organism:
    def __init__(self,
                 data: Dict[str,tf.data.Dataset],
                 config=None,
                 chromosome={},
                 phase=0,
                 prevBestOrganism=None):
        '''
        config is a . accessible dict object containing model params that will stay constant during evolution
        chromosome is a dictionary of genes
        phase is the phase that the individual belongs to
        prevBestOrganism is the best organism of the previous phase
        
        TODO:
        
        1. implement to_json and from_json methods for copies
        2. Separate out step where organism is associated with a dataset
        '''
        self.data = data
        self.train_data = data['train']
        self.val_data = data['val']
        self.test_data = data['test']
        self.config = config
        self.phase = phase
        self.chromosome = chromosome
        self.prevBestOrganism=prevBestOrganism
        if phase != 0:
            # In a later stage, the model is made by
            # attaching new layers to the prev best model
            self.last_model = prevBestOrganism.model
    
    @property
    def config(self):
        return self._config
    
    @config.setter
    def config(self, config=None):
        config = config or OmegaConf.create({})
        config.input_shape = config.input_shape or (224,224,3)
        config.output_size = config.output_size or 38
        config.epochs_per_organism = config.epochs_per_organism or 5
        self._config = config
    
    def build_model(self):
        '''
        This is the function to build the keras model
        '''
        K.clear_session()
        inputs = Input(shape=self.config.input_shape)
        if self.phase != 0:
            # Slice the prev best model
            # Use the model as a layer
            # Attach new layer to the sliced model
            intermediate_model = Model(inputs=self.last_model.input,
                                       outputs=self.last_model.layers[-3].output)
            for layer in intermediate_model.layers:
                # To make the iteration efficient
                layer.trainable = False
            inter_inputs = intermediate_model(inputs)
            x = Conv2D(filters=self.chromosome['a_output_channels'],
                       padding='same',
                       kernel_size=self.chromosome['a_filter_size'],
                       use_bias=self.chromosome['a_include_BN'])(inter_inputs)
            # This is to ensure that we do not randomly chose anothere activation
            self.chromosome['activation_type'] = self.prevBestOrganism.chromosome['activation_type']
        else:
            # For PHASE 0 only
            # input layer
            x = Conv2D(filters=self.chromosome['a_output_channels'],
                       padding='same',
                       kernel_size=self.chromosome['a_filter_size'],
                       use_bias=self.chromosome['a_include_BN'])(inputs)
        if self.chromosome['a_include_BN']:
            x = BatchNormalization()(x)
        x = self.chromosome['activation_type']()(x)
        if self.chromosome['include_pool']:
            x = self.chromosome['pool_type'](strides=(1,1),
                                             padding='same')(x)
        if self.phase != 0 and self.chromosome['include_layer'] == False:
            # Except for PHASE0, there is a choice for
            # the number of layers that the model wants
            if self.chromosome['include_skip']:
                y = Conv2D(filters=self.chromosome['a_output_channels'],
                           kernel_size=(1,1),
                           padding='same')(inter_inputs)
                x = Add()([y,x])
            x = GlobalAveragePooling2D()(x)
            x = Dense(self.output_shape, activation='softmax')(x)
        else:
            # PHASE0 or no skip
            # in the tail
            x = Conv2D(filters=self.chromosome['b_output_channels'],
                       padding='same',
                       kernel_size=self.chromosome['b_filter_size'],
                       use_bias=self.chromosome['b_include_BN'])(x)
            if self.chromosome['b_include_BN']:
                x = BatchNormalization()(x)
            x = self.chromosome['activation_type']()(x)
            if self.chromosome['include_skip']:
                y = Conv2D(filters=self.chromosome['b_output_channels'],
                           padding='same',
                           kernel_size=(1,1))(inputs)
                x = Add()([y,x])
            x = GlobalAveragePooling2D()(x)
            x = Dense(self.config.output_size, activation='softmax')(x)
        self.model = Model(inputs=[inputs], outputs=[x])
        self.model.compile(optimizer='adam',
                           loss='categorical_crossentropy',
                           metrics=['accuracy'])
        
    def fitnessFunction(self,
                        train_data,
                        val_data,
                        generation_number):
        '''
        This function is used to calculate the
        fitness of an individual.
        '''
        wandb.init(entity="jrose",
                   project=f"vlga-plant_village",
                   group='KAGp{}'.format(self.phase),
                   job_type='g{}'.format(generation_number))
        
        self.model.fit(train_data,
                       epochs=self.config.epochs_per_organism,
                       callbacks=[WandbCallback()],
                       verbose=1)
        _, self.fitness = self.model.evaluate(val_data,
                                              verbose=1)
    def crossover(self,
                  partner,
                  generation_number):
        '''
        This function helps in making children from two
        parent individuals.
        '''
        child_chromosome = {}
        endpoint = np.random.randint(low=0, high=len(self.chromosome))
        for idx, key in enumerate(self.chromosome):
            if idx <= endpoint:
                child_chromosome[key] = self.chromosome[key]
            else:
                child_chromosome[key] = partner.chromosome[key]
        child = Organism(chromosome=child_chromosome,
                         data=self.data,
                         config=self.config,
                         phase=self.phase,
                         prevBestOrganism=self.prevBestOrganism)
        child.build_model()
        child.fitnessFunction(self.train_data,
                              self.val_data,
                              generation_number=generation_number)
        return child
    
    def mutation(self, generation_number):
        '''
        One of the gene is to be mutated.
        '''
        index = np.random.randint(0, len(self.chromosome))
        key = list(self.chromosome.keys())[index]
        if  self.phase != 0:
            self.chromosome[key] = options[key][np.random.randint(len(options[key]))]
        else:
            self.chromosome[key] = options_phase0[key][np.random.randint(len(options_phase0[key]))]
        self.build_model()
        self.fitnessFunction(self.train_data,
                             self.val_data,
                             generation_number=generation_number)
    
    def show(self):
        '''
        Util function to show the individual's properties.
        '''
        pp.pprint(self.config)
        pp.pprint(self.chromosome)
        
    


In [7]:
def random_hyper(phase):
    if phase == 0:
        return {
        'a_filter_size': options_phase0['a_filter_size'][np.random.randint(len(options_phase0['a_filter_size']))],
        'a_include_BN': options_phase0['a_include_BN'][np.random.randint(len(options_phase0['a_include_BN']))],
        'a_output_channels': options_phase0['a_output_channels'][np.random.randint(len(options_phase0['a_output_channels']))],
        'activation_type': options_phase0['activation_type'][np.random.randint(len(options_phase0['activation_type']))],
        'b_filter_size': options_phase0['b_filter_size'][np.random.randint(len(options_phase0['b_filter_size']))],
        'b_include_BN': options_phase0['b_include_BN'][np.random.randint(len(options_phase0['b_include_BN']))],
        'b_output_channels': options_phase0['b_output_channels'][np.random.randint(len(options_phase0['b_output_channels']))],
        'include_pool': options_phase0['include_pool'][np.random.randint(len(options_phase0['include_pool']))],
        'pool_type': options_phase0['pool_type'][np.random.randint(len(options_phase0['pool_type']))],
        'include_skip': options_phase0['include_skip'][np.random.randint(len(options_phase0['include_skip']))]
        }
    else:
        return {
        'a_filter_size': options['a_filter_size'][np.random.randint(len(options['a_filter_size']))],
        'a_include_BN': options['a_include_BN'][np.random.randint(len(options['a_include_BN']))],
        'a_output_channels': options['a_output_channels'][np.random.randint(len(options['a_output_channels']))],
        'b_filter_size': options['b_filter_size'][np.random.randint(len(options['b_filter_size']))],
        'b_include_BN': options['b_include_BN'][np.random.randint(len(options['b_include_BN']))],
        'b_output_channels': options['b_output_channels'][np.random.randint(len(options['b_output_channels']))],
        'include_pool': options['include_pool'][np.random.randint(len(options['include_pool']))],
        'pool_type': options['pool_type'][np.random.randint(len(options['pool_type']))],
        'include_layer': options['include_layer'][np.random.randint(len(options['include_layer']))],
        'include_skip': options['include_skip'][np.random.randint(len(options['include_skip']))]
        }

In [8]:
def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()

# Generation
This is a class that hold generations of models.

1. fitSurvivalRate - The amount of fit individuals we want in the next generation.
2. unfitSurvivalProb - The probability of sending unfit individuals
3. mutationRate - The mutation rate to change genes in an individual.
4. phase - The phase that the generation belongs to.
5. population_size - The amount of individuals that the generation consists of.
6. prevBestOrganism - The best organism (individual) is the last phase

In [9]:
class Generation:
    def __init__(self,
                 data,
                 config,
                 fitSurvivalRate,
                 unfitSurvivalProb,
                 mutationRate,
                 phase,
                 population_size,
                 prevBestOrganism):
        self.data = data
        self.config = config
#         self.population_size = population_size
        self.population = []
        self.generation_number = 0
#         self.mutationRate = mutationRate
#         self.fitSurvivalRate = fitSurvivalRate
#         self.unfitSurvivalProb = unfitSurvivalProb
#         self.prevBestOrganism = prevBestOrganism
        self.phase = phase
        # creating the first population: GENERATION_0
        # can be thought of as the setup function
        
        self.initialize_population()
        
    @property
    def config(self):
        return self._config
    
    @config.setter
    def config(self, config=None):
        config = config or OmegaConf.create({})
        config.population_size = config.population_size or 5
        config.num_generations_per_phase = config.num_generations_per_phase or 3
        config.fitSurvivalRate = config.fitSurvivalRate or 0.5
        config.unfitSurvivalProb = config.unfitSurvivalProb or 0.2
        config.mutationRate = config.mutationRate or 0.1
        config.num_phases = config.num_phases or 5
        
        self._config = config
        self.__dict__.update(config)
        
        
    def initialize_population(self):
        '''
        1. Create self.population_size individual organisms from scratch by randomly sampling an initial set of hyperparameters (a chromosome)
        2. As each is instantiated, build its model
        3. Assess their fitness one-by-one
        4. Sort models by relative fitness so we have a (potentially) new Best Organism (best model)
        4. Increment generation number to 1
        '''

        for idx in range(self.population_size):
            print(f'Creating, training then testing organism {idx} of generation {self.generation_number} and phase {self.phase}')
            org = Organism(chromosome=random_hyper(self.phase),
                           data=self.data,
                           config=self.config,
                           phase=self.phase,
                           prevBestOrganism=self.prevBestOrganism)
            org.build_model()
            org.fitnessFunction(org.data['train'],
                                org.data['test'],
                                generation_number=self.generation_number)
            self.population.append(org)

        # sorts the population according to fitness (high to low)
        self.sortModel()
        self.generation_number += 1

    def sortModel(self):
        '''
        sort the models according to the 
        fitness in descending order.
        '''
        fitness = [ind.fitness for ind in self.population]
        sort_index = np.argsort(fitness)[::-1]
        self.population = [self.population[index] for index in sort_index]

    def generate(self):
        '''
        Generate a new generation in the same phase
        '''
        number_of_fit = int(self.population_size * self.fitSurvivalRate)
        new_pop = self.population[:number_of_fit]
        for individual in self.population[number_of_fit:]:
            if np.random.rand() <= self.unfitSurvivalProb:
                new_pop.append(individual)
        for index, individual in enumerate(new_pop):
            if np.random.rand() <= self.mutationRate:
                new_pop[index].mutation(generation_number=self.generation_number)
        fitness = [ind.fitness for ind in new_pop]
        children=[]
        for idx in range(self.population_size-len(new_pop)):
            parents = np.random.choice(new_pop, replace=False, size=(2,), p=softmax(fitness))
            A=parents[0]
            B=parents[1]
            child=A.crossover(B, generation_number=self.generation_number)
            children.append(child)
        self.population = new_pop+children
        self.sortModel()
        self.generation_number+=1

    def evaluate(self, last=False):
        '''
        Evaluate the generation
        '''
        fitness = [ind.fitness for ind in self.population]
        wandb.log({'Best fitness': fitness[0]})
        wandb.log({'Average fitness': sum(fitness)/len(fitness)})
        
        self.population[0].show()
        if last:
            return self.population[0]
        
        


In [12]:
# population_size = 5
# num_generations_per_phase = 3
# fitSurvivalRate = 0.5
# unfitSurvivalProb = 0.2
# mutationRate = 0.1
# num_phases = 5
# prevBestOrganism = None

generation_config = OmegaConf.create({
                                      'population_size':5,
                                      'num_generations_per_phase':3,
                                      'fitSurvivalRate': 0.5,
                                      'unfitSurvivalProb':0.2,
                                      'mutationRate':0.1,
                                      'num_phases':5
                                    })
prevBestOrganism = None



for phase in range(num_phases):
    print("PHASE {}".format(phase))
    generation = Generation(data=data,
                            config=config,
                            fitSurvivalRate=fitSurvivalRate,
                            unfitSurvivalProb=unfitSurvivalProb,
                            mutationRate=mutationRate,
                            population_size=population_size,
                            phase=phase,
                            prevBestOrganism=prevBestOrganism)
#     while generation.generation_number < num_generations_per_phase:
    generation.generate()
    if generation.generation_number == num_generations_per_phase:
        # Last generation is the phase
        # print('I AM THE BEST IN THE PHASE')
        prevBestOrganism = generation.evaluate(last=True)
        model_path = f'best-model-phase_{phase}.png'
        tf.keras.utils.plot_model(prevBestOrganism.model, to_file=model_path)
        wandb.log({"best_model": [wandb.Image(model_path, caption=f"Best Model phase_{phase}")]})
        log_high_loss_examples(prevBestOrganism.test_dataset,
                               prevBestOrganism.model, 
                               k=32)
    else:
        generation.evaluate()

PHASE 0
Creating, training then testing organism 0 of generation 0 and phase 0


ERROR:wandb.jupyter:Failed to query for notebook name, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable
wandb: Currently logged in as: jrose (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.10.11 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


Epoch 1/3
2037/2037 [==============================] - 469s 230ms/step - loss: 1.8262 - accuracy: 0.4844
Epoch 2/3
2037/2037 [==============================] - 469s 230ms/step - loss: 1.3346 - accuracy: 0.6083
Epoch 3/3
1019/1019 [==============================] - 96s 95ms/step - loss: 1.6317 - accuracy: 0.5230
Creating, training then testing organism 1 of generation 0 and phase 0


wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


epoch,2
loss,1.17093
accuracy,0.65469
_step,2
_runtime,1422
_timestamp,1606324931


...

epoch,▁▅█
loss,█▃▁
accuracy,▁▆█
_step,▁▅█
_runtime,▁▅█
_timestamp,▁▅█


wandb: wandb version 0.10.11 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


Epoch 1/3
   2/2037 [..............................] - ETA: 3:08 - loss: 3.9250 - accuracy: 0.0000e+00WARNING:tensorflow:Callbacks method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0666s vs `on_train_batch_end` time: 0.1163s). Check your callbacks.


2037/2037 [==============================] - 364s 179ms/step - loss: 2.5563 - accuracy: 0.3154
Epoch 2/3
2037/2037 [==============================] - 364s 179ms/step - loss: 2.0210 - accuracy: 0.4357
Epoch 3/3
340/340 [==============================] - 21s 63ms/step - loss: 2.6488 - accuracy: 0.3076
{'seed': 237, 'batch_size': 16, 'input_shape': [224, 224, 3], 'output_size': 38, 'epochs_per_organism': 3}
{   'a_filter_size': (5, 5),
    'a_include_BN': True,
    'a_output_channels': 256,
    'activation_type': <class 'tensorflow.python.keras.layers.advanced_activations.ELU'>,
    'b_filter_size': (9, 9),
    'b_include_BN': False,
    'b_output_channels': 128,
    'include_pool': False,
    'include_skip': True,
    'pool_type': <class 'tensorflow.python.keras.layers.pooling.AveragePooling2D'>}
('Failed to import pydot. You must `pip install pydot` and install graphviz (https://graphviz.gitlab.io/download/), ', 'for `pydotprint` to work.')


FileNotFoundError: [Errno 2] No such file or directory: 'best.png'

In [ ]:
### 1. Using tfds.features.ClassLabel

# feature_labels = tfds.features.ClassLabel(names=vocab)
# data = ['Potato___healthy',
#         'Potato___Late_blight',
#         'Raspberry___healthy',
#         'Soybean___healthy',
#         'Squash___Powdery_mildew',
#         'Strawberry___healthy',
#         'Strawberry___Leaf_scorch',
#         'Tomato___Bacterial_spot',
#         'Tomato___Early_blight',
#         'Tomato___healthy']

# data += data[::-1]
# print([feature_labels.str2int(label) for label in data])
# data = train_data
# data_enc = data.map(lambda x,y: (x, feature_labels.int2str(y)))

### 2. Using StringLookup and CategoryEncoding Layers

# layer = StringLookup(vocabulary=vocab, num_oov_indices=0, mask_token=None)
# i_layer = StringLookup(vocabulary=layer.get_vocabulary(), invert=True)
# int_data = layer(data)

# print(len(layer.get_vocabulary()))
# print(len(class_encoder.class_list))
# print(set(layer.get_vocabulary())==set(class_encoder.class_list))

# i_layer = StringLookup(vocabulary=layer.get_vocabulary(), invert=True)
# int_data = layer(data)

# print(layer(data))
# print(i_layer(int_data))

In [ ]:
# # from tensorflow.keras.layers.experimental.preprocessing import StringLookup, CategoryEncoding
# # data = tf.constant(["a", "b", "c", "b", "c", "a"])
# # # Use StringLookup to build an index of the feature values
# # indexer = StringLookup()
# # indexer.adapt(data)
# # # Use CategoryEncoding to encode the integer indices to a one-hot vector
# # encoder = CategoryEncoding(output_mode="binary")
# # encoder.adapt(indexer(data))
# # # Convert new test data (which includes unknown feature values)
# # test_data = tf.constant(["a", "b", "c", "d", "e", ""])
# # encoded_data = encoder(indexer(test_data))
# # print(encoded_data)

# vocab = ["a", "b", "c", "d"]
# data = tf.constant([["a", "c", "d"], ["d", "z", "b"]])
# layer = StringLookup(vocabulary=vocab)
# i_layer = StringLookup(vocabulary=layer.get_vocabulary(), invert=True)
# int_data = layer(data)

# print(layer(data))
# print(i_layer(int_data))